In [1]:
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
from PIL import Image
import json
import random
import fastwer
from tqdm.auto import tqdm
import os

2025-02-20 09:32:51.518527: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740065571.538784  801509 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740065571.543369  801509 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-20 09:32:51.564890: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
CKP_MODEL = "../finetuned/trocr/Seth_Lowell/V_1" #"/home/ralvarez22/Documentos/llm_data/llm_cache/models--microsoft--trocr-large-stage1/snapshots/3c8ead8dfda428d914334169380bb546f770a300"
DEVICE = "cuda" # Where to load the model
DATASET_PATH = "../hand-cursive-trocr"
DATASET_FILE = "../hand-cursive-trocr/valid_metadata.json"

In [3]:
processor = TrOCRProcessor.from_pretrained(CKP_MODEL,device_map=DEVICE)
model = VisionEncoderDecoderModel.from_pretrained(CKP_MODEL, device_map=DEVICE)
# This configuration allows to correctly set the BOS token for inference and the Temperature and Sample strategy for the decoder
# This could be setted on the generation_config.json file, generated when saving the model, but, to avoid mismatch or errors, I overwrite this
model.generation_config.decoder_start_token_id = processor.tokenizer.bos_token_id
model.generation_config.temperature = 0.4
model.generation_config.do_sample = True

Config of the encoder: <class 'transformers.models.vit.modeling_vit.ViTModel'> is overwritten by shared encoder config: ViTConfig {
  "attention_probs_dropout_prob": 0.0,
  "bos_token_id": 0,
  "decoder_start_token_id": 0,
  "encoder_stride": 16,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 1024,
  "image_size": 384,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-12,
  "model_type": "vit",
  "num_attention_heads": 16,
  "num_channels": 3,
  "num_hidden_layers": 24,
  "patch_size": 16,
  "qkv_bias": false,
  "transformers_version": "4.46.3"
}

Config of the decoder: <class 'transformers.models.trocr.modeling_trocr.TrOCRForCausalLM'> is overwritten by shared decoder config: TrOCRConfig {
  "activation_dropout": 0.0,
  "activation_function": "relu",
  "add_cross_attention": true,
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classifier_dropout": 0.0,
  "cross_attention_hidden_size": 1024,
  "d_model": 1

In [4]:
cropped_images_dataset = json.load(open(DATASET_FILE, "r"))

In [5]:
#items_to_eval = int(len(cropped_images_dataset) * 0.9)

In [6]:
#items_to_eval

In [7]:
#test_dataset = random.sample(cropped_images_dataset, items_to_eval)

In [8]:
cer_value = 0
wer_value = 0
for test_item in tqdm(cropped_images_dataset):
    pil_image = Image.open(os.path.join(DATASET_PATH, test_item["image"])).convert("RGB")
    proc_image = processor(pil_image, return_tensors="pt").pixel_values.to(DEVICE)
    tgt_text = test_item["label"]
    data_generated = model.generate(proc_image)
    gen_text = processor.tokenizer.decode(data_generated[0].cpu(), skip_special_tokens=True)
    #print("Original: {} - Recognized: {}".format(tgt_text, gen_text))
    cer_val = fastwer.score_sent(gen_text, tgt_text, char_level=True)
    wer_val = fastwer.score_sent(gen_text, tgt_text)
    cer_value += cer_val
    wer_value += wer_val

  0%|          | 0/26728 [00:00<?, ?it/s]

In [9]:
cer_value /= len(cropped_images_dataset)
wer_value /= len(cropped_images_dataset)

In [10]:
print("CER Avg value: {:.4f}".format(cer_value))
print("WER Avg value: {:.4f}".format(wer_value))

CER Avg value: 4.1046
WER Avg value: 7.3069
